<a href="https://colab.research.google.com/github/anajuliabit/desafio-2-2020/blob/master/parte_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MARATONA BEHIND THE CODE 2020

## DESAFIO 2: PARTE 2

### Introdução

Na parte 1 deste desafio, você realizou o pré-processamento e o treinamento de um modelo a partir de um conjunto de dados base fornecido. Nesta segunda etapa você irá integrar todas as transformações e eventos de treinamento criados anteriormente em uma Pipeline completa para *deploy* no **Watson Machine Learning**!

### Preparação do Notebook

Primeiro realizaremos a instalação do scikit-learn e a importação das mesmas bibliotecas utilizadas anteriormente

In [23]:
!pip install scikit-learn==0.20.0 --upgrade

Requirement already up-to-date: scikit-learn==0.20.0 in /usr/local/lib/python3.6/dist-packages (0.20.0)


In [24]:
import json
import requests
import numpy as np
import pandas as pd
import xgboost as xgb
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.pipeline import Pipeline
from sklearn.model_selection import KFold, cross_validate
from imblearn.over_sampling import SMOTE

É necessário inserir o conjunto de dados base novamente como um dataframe pandas, seguindo as instruções

![alt text](https://i.imgur.com/K1DwL9I.png "importing-csv-as-df")

Após a seleção da opção **"Insert to code"**, a célula abaixo será preenchida com o código necessário para importação e leitura dos dados no arquivo .csv como um DataFrame Pandas.

In [25]:
# import types
# import pandas as pd
# from botocore.client import Config
# import ibm_boto3

# def __iter__(self): return 0

# # @hidden_cell
# # The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# # You might want to remove those credentials before you share the notebook.
# client_a476a5a683ec4d47b59cd07c271eccfd = ibm_boto3.client(service_name='s3',
#     ibm_api_key_id='UF8UfW1d8VBFUry-ptPC-t-Sxf7zR5FVgnRwD0pDCUqS',
#     ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
#     config=Config(signature_version='oauth'),
#     endpoint_url='https://s3.eu-geo.objectstorage.service.networklayer.com')

# body = client_a476a5a683ec4d47b59cd07c271eccfd.get_object(Bucket='desafio2-donotdelete-pr-ynhjpdfnvjzy0b',Key='dataset_desafio_2.csv')['Body']
# # add missing __iter__ method, so pandas accepts body as file-like object
# if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

df_data_2 = pd.read_csv("dataset_desafio_2.csv")
df_data_2.head()


,MATRICULA,NOME,REPROVACOES_DE,REPROVACOES_EM,REPROVACOES_MF,REPROVACOES_GO,NOTA_DE,NOTA_EM,NOTA_MF,NOTA_GO,INGLES,H_AULA_PRES,TAREFAS_ONLINE,FALTAS,PERFIL
0,502375,Márcia Illiglener,0,0,0,0,6.2,5.8,4.6,5.9,0.0,2,4,3,EXATAS
1,397093,Jason Jytereoman Izoimum,0,0,0,0,6.0,6.2,5.2,4.5,1.0,2,4,3,EXATAS
2,915288,Bartolomeu Inácio da Gama,0,0,0,0,7.3,6.7,7.1,7.2,0.0,5,0,3,HUMANAS
3,192652,Fernanda Guedes,1,3,1,1,0.0,0.0,0.0,0.0,1.0,4,4,4,DIFICULDADE
4,949491,Alessandre Borba Gomes,1,3,1,1,0.0,0.0,0.0,0.0,1.0,5,2,5,DIFICULDADE


### Construção da Pipeline completa para encapsulamento no WML

#### Preparando transformações personalizadas para carregamento no WML

Na etapa anterior, foi mostrado como criar uma transformação personalizada, através da declaração de uma classe Python com os métodos ``fit`` e ``transform``.

    - Código da transformação personalizada DropColumns():
    
    from sklearn.base import BaseEstimator, TransformerMixin
    # All sklearn Transforms must have the `transform` and `fit` methods
    class DropColumns(BaseEstimator, TransformerMixin):
        def __init__(self, columns):
            self.columns = columns
        def fit(self, X, y=None):
            return self
        def transform(self, X):
            # Primeiro realizamos a cópia do dataframe 'X' de entrada
            data = X.copy()
            # Retornamos um novo dataframe sem as colunas indesejadas
            return data.drop(labels=self.columns, axis='columns')

Para integrar esses tipos de transformações personalizadas nas Pipelines do Watson Machine Learning, é necessário primeiramente empacotar seu código personalizado como uma biblioteca Python. Isso pode ser feito facilmente com o uso da ferramenta *setuptools*.

No seguinte repositório git: https://github.com/vnderlev/sklearn_transforms temos todos os arquivos necessários para a criação de um pacote Python, nomeado **my_custom_sklearn_transforms**.
Esse pacote possui a seguinte estrutura de arquivos:

    /my_custom_sklearn_transforms.egg-info
        dependency_links.txt
        not-zip-safe
        PKG-INFO
        SOURCES.txt
        top_level.txt
    /my_custom_sklearn_transforms
        __init__.py
        sklearn_transformers.py
    PKG-INFO
    README.md
    setup.cfg
    setup.py
    
O arquivo principal, que irá conter o código das nossas transformadas personalizadas, é o arquivo **/my_custom_sklearn_transforms/sklearn_transformers.py**. Se você acessá-lo no repositório, irá notar que ele contém exatamente o mesmo código declarado na primeira etapa (a classe DropColumns).

Caso você tenha declarado transformações próprias (além da DropColumn fornecida), você deverá adicionar todas as classes dessas transformadas criadas por você nesse mesmo arquivo. Para tal, você deve realizar o fork desse repositório (isso pode ser feito na própria interface Web do Github, clicando no botão conforme a imagem abaixo), e adicionar suas classes personalizadas no arquivo **sklearn_transformers.py**.

![alt text](https://i.imgur.com/D81E1uM.png "forking-a-repo")

Se você somente fez o uso da transformação fornecida (DropColumns), pode ignorar essa etapa de fork, e seguir utilizando o pacote base fornecido! :)

Após a preparação do seu pacote Python com as suas transformadas personalizadas, substitua o link do repositório git na célula abaixo e execute-a. Caso você não tenha preparado nenhuma nova transformada, execute a célula com o link do repositório já fornecido. 

<hr>
    
**OBSERVAÇÃO**

Caso a execução da célula abaixo retorne um erro de que o repositório já existe, execute:

**!rm -r -f sklearn_transforms**

In [26]:
# substitua o link abaixo pelo link do seu repositório git (se for o caso)

# !rm -r -f sklearn_transforms
!git clone https://github.com/anajuliabit/sklearn_transforms

fatal: destination path 'sklearn_transforms' already exists and is not an empty directory.


In [ ]:
!cd sklearn_transforms
!ls -ltr

total 1476
drwxr-xr-x 1 root root    4096 Jul 30 16:30 sample_data
-rw-r--r-- 1 root root 1500234 Aug 20 17:05 dataset_desafio_2.csv
drwxr-xr-x 6 root root    4096 Aug 20 17:05 sklearn_transforms


Para subir o código no WML, precisamos enviar um arquivo .zip com todo o código fonte, então iremos zipar o diretório clonado em seguida:

In [27]:
!zip -r sklearn_transforms.zip sklearn_transforms

updating: sklearn_transforms/ (stored 0%)
updating: sklearn_transforms/README.md (deflated 15%)
updating: sklearn_transforms/.git/ (stored 0%)
updating: sklearn_transforms/.git/objects/ (stored 0%)
updating: sklearn_transforms/.git/objects/info/ (stored 0%)
updating: sklearn_transforms/.git/objects/pack/ (stored 0%)
updating: sklearn_transforms/.git/objects/pack/pack-c2a60ffc4ca14000037e35dcad24efb6909ccc40.pack (deflated 3%)
updating: sklearn_transforms/.git/objects/pack/pack-c2a60ffc4ca14000037e35dcad24efb6909ccc40.idx (deflated 19%)
updating: sklearn_transforms/.git/packed-refs (deflated 11%)
updating: sklearn_transforms/.git/info/ (stored 0%)
updating: sklearn_transforms/.git/info/exclude (deflated 28%)
updating: sklearn_transforms/.git/hooks/ (stored 0%)
updating: sklearn_transforms/.git/hooks/pre-commit.sample (deflated 43%)
updating: sklearn_transforms/.git/hooks/pre-push.sample (deflated 50%)
updating: sklearn_transforms/.git/hooks/post-update.sample (deflated 27%)
updating: sk

Com o arquivo zip do nosso pacote carregado no Kernel deste notebook, podemos utilizar a ferramenta pip para instalá-lo, conforme a célula abaixo:

In [28]:
!pip install sklearn_transforms.zip

Processing ./sklearn_transforms.zip
  Created wheel for my-custom-sklearn-transforms: filename=my_custom_sklearn_transforms-1.0-cp36-none-any.whl size=2205 sha256=797848b956103b28d3d5908e22d889219ba8041923f403506ce39d0b5e55b580
  Stored in directory: /tmp/pip-ephem-wheel-cache-jrmph8rn/wheels/c4/57/de/7704133573948922952f7658fc7094c1d36d9b79a4224af702
Successfully built my-custom-sklearn-transforms
  Found existing installation: my-custom-sklearn-transforms 1.0
    Uninstalling my-custom-sklearn-transforms-1.0:
      Successfully uninstalled my-custom-sklearn-transforms-1.0


Podemos agora realizar a importação do nosso pacote personalizado em nosso notabook!

Iremos importar a transformação DropColumns. Se você possui outras transformações personalizadas, não se esqueça de importá-las!

In [29]:
from my_custom_sklearn_transforms.sklearn_transformers import DropColumns
from my_custom_sklearn_transforms.sklearn_transformers import SmoteResample

#### Declarando a Pipeline

Após a importação das transformações personalizadas como um pacote Python, podemos partir para a declaração da nossa Pipeline.

O processo é bem semelhante ao realizado na primeira etapa, porém com algumas diferenças importantes, então preste bem atenção!

A Pipeline exemplo possui três estágios: 

    - remover a coluna "NOME"
    - imputar "zeros" em todos os valores faltantes
    - inserir os dados pré-processados como entrada em um modelo treinado
    
Relembrando, a entrada desta Pipeline será o conjunto cru de dados fornecido exceto a coluna "LABELS" (variável-alvo a ser determinada pelo modelo).

Teremos então 17 valores de entrada **na PIPELINE** (no modelo serão 16 entradas, pois a coluna NAME será removida no primeiro estágio após a transformação DropColumn).

    MATRICULA       - número de quatro algarismos único para cada estudante
    NOME            - nome completo do estudante
    FALTAS_DE       - número de faltas na disciplina de ``Direito Empresarial``
    FALTAS_EM       - número de faltas na disciplina de ``Empreendedorismo``
    FALTAS_MF       - número de faltas na disciplina de ``Matemática Financeira``
    MEDIA_DE        - média simples das notas do aluno na disciplina de ``Direito Empresarial`` (0-10)
    MEDIA_EM        - média simples das notas do aluno na disciplina de ``Empreendedorismo`` (0-10)
    MEDIA_MF        - média simples das notas do aluno na disciplina de ``Matemática Financeira`` (0-10)
    HRS_ESTUDO_DE   - horas de estudo particular na disciplina de ``Direito Empresarial``
    HRS_ESTUDO_EM   - horas de estudo particular na disciplina de ``Empreendedorismo``
    HRS_ESTUDO_MF   - horas de estudo particular na disciplina de ``Matemática Financeira``
    REPROVACOES_DE  - número de reprovações na disciplina de ``Direito Empresarial``
    REPROVACOES_EM  - número de reprovações na disciplina de ``Empreendedorismo``
    REPROVACOES_MF  - número de reprovações na disciplina de ``Matemática Financeira``
    LIVROS_TEXTO    - quantidade de livros e textos acessados pelo aluno no sistema da universidade
    AULAS_AO_VIVO   - horas de aulas ao vivo presenciadas pelo aluno (total em todas as disciplinas)
    EXERCICIOS      - número de exercícios realizados pelo estudante (total em todas as disciplinas) no sistema da universidade

A saída da Pipeline será um valor estimado para a coluna "LABELS".

In [30]:
# Criação de uma Transform personalizada ``DropColumns``

rm_columns = DropColumns(
    columns=["NOME"]
)

In [31]:
# Criação de um objeto ``SimpleImputer``

si = SimpleImputer(
    missing_values=np.nan,  # os valores faltantes são do tipo ``np.nan`` (padrão Pandas)
    strategy='constant',  # a estratégia escolhida é a alteração do valor faltante por uma constante
    fill_value=-1,  # a constante que será usada para preenchimento dos valores faltantes é um int64=0.
    verbose=0,
    copy=True
)

In [32]:
df = df_data_2.replace(np.nan, -1)

In [54]:
# Definição das colunas que serão features (nota-se que a coluna NOME não está presente)
features = [
    'MATRICULA', 'REPROVACOES_DE', 'REPROVACOES_EM', "REPROVACOES_MF", "REPROVACOES_GO",
    "NOTA_DE", "NOTA_EM", "NOTA_MF", "NOTA_GO",
    "INGLES", "H_AULA_PRES", "TAREFAS_ONLINE", "FALTAS", 
]


# Definição da variável-alvo
target = ["PERFIL"]

# Preparação dos argumentos para os métodos da biblioteca scikit-learn
X = df[features]
y = df[target]

X_balanced,y_balanced = SmoteResample().fit(X,y)

X.insert(1, 'NOME', 'Ana')
X_balanced.insert(1, 'NOME', 'Ana')
X_balanced.MATRICULA = X_balanced.MATRICULA.astype(int)

/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:752: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [55]:
X_balanced.head()

,MATRICULA,NOME,REPROVACOES_DE,REPROVACOES_EM,REPROVACOES_MF,REPROVACOES_GO,NOTA_DE,NOTA_EM,NOTA_MF,NOTA_GO,INGLES,H_AULA_PRES,TAREFAS_ONLINE,FALTAS
0,502375,Ana,0.0,0.0,0.0,0.0,6.2,5.8,4.6,5.9,0.0,2.0,4.0,3.0
1,397093,Ana,0.0,0.0,0.0,0.0,6.0,6.2,5.2,4.5,1.0,2.0,4.0,3.0
2,915288,Ana,0.0,0.0,0.0,0.0,7.3,6.7,7.1,7.2,0.0,5.0,0.0,3.0
3,192652,Ana,1.0,3.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,4.0,4.0,4.0
4,949491,Ana,1.0,3.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,5.0,2.0,5.0


**ATENÇÃO!!**

A célula acima, embora muito parecida com a definição de features na primeira etapa deste desafio, possui uma grande diferença!

Nela está presente a coluna "NOME" como uma feature! Isso ocorre pois neste caso essas são as entradas da *PIPELINE*, e não do modelo.

In [47]:
# Separação dos dados em um conjunto de treino e um conjunto de teste
X_train, X_test, y_train, y_test = train_test_split(X_balanced, y_balanced)

# Separação dos dados sem balanceamento
X_treino, X_teste, y_treino, y_teste = train_test_split(X, y)

Na célula abaixo é realizada a declaração de um objeto **Pipeline** do scikit-learn, onde é declarado o parâmetro *steps*, que nada mais é do que uma lista com as etapas da nossa pipeline:

    'remove_cols'     - transformação personalizada DropColumns
    'imputer'         - transformação embutida do scikit-learn para imputação de valores faltantes
    'set_index'       - seta o index
    'dtc'             - um classificador via árvore de decisão
    
Note que passamos como passos as transformadas instanciadas anteriormente, sob nome `rm_columns` e `si`.

In [48]:
# Criação da nossa pipeline para armazenamento no Watson Machine Learning:
my_pipeline = Pipeline(
    steps=[
        ('remove_cols', rm_columns),
        ('imputer', si),
        ('dtc', ExtraTreesClassifier(random_state=42)),
    ]
)

Em seguida iremos executar o método `fit()` da Pipeline, realizando o pré-processamento e o treinamento do modelo de uma só vez.

In [49]:
# Inicialização da Pipeline (pré-processamento e realização do treinamento do modelo)
my_pipeline.fit(X_train, y_train)

/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/forest.py:248: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


Pipeline(memory=None,
     steps=[('remove_cols', DropColumns(columns=['NOME'])), ('imputer', SimpleImputer(copy=True, fill_value=-1, missing_values=nan,
       strategy='constant', verbose=0)), ('dtc', ExtraTreesClassifier(bootstrap=False, class_weight=None, criterion='gini',
           max_depth=None, max_features='auto', m...imators=10, n_jobs=None,
           oob_score=False, random_state=42, verbose=0, warm_start=False))])

In [50]:
y_pred = my_pipeline.predict(X_teste)

In [51]:

from sklearn.metrics import accuracy_score

# Acurácia alcançada pela árvore de decisão
print("Acurácia: {}%".format(100*round(accuracy_score(y_teste, y_pred), 2)))

Acurácia: 95.0%


Agora que temos uma pipeline completa, com etapas de pré-processamento configuradas e também um modelo por árvore de decisão já treinado, podemos realizar a integração com o Watson Machine Learning!

### Encapsulando uma Pipeline personalizada no Watson Machine Learning

#### Estabelecendo conexão entre o cliente Python do WML e a sua instância do serviço na nuvem

In [ ]:
# Biblioteca Python com implementação de um cliente HTTP para a API do WML
from watson_machine_learning_client import WatsonMachineLearningAPIClient

2020-08-18 19:12:27,661 - watson_machine_learning_client.wml_client_error - WARNING - Failure during creation of library.
Reason: (400)
Reason: Bad Request
HTTP response headers: HTTPHeaderDict({'Server': 'nginx', 'Date': 'Tue, 18 Aug 2020 19:12:27 GMT', 'Content-Type': 'application/json', 'Content-Length': '214', 'Connection': 'keep-alive', 'X-Frame-Options': 'DENY', 'X-Content-Type-Options': 'nosniff', 'X-XSS-Protection': '1', 'Pragma': 'no-cache', 'Cache-Control': 'private, no-cache, no-store, must-revalidate', 'X-WML-User-Client': 'PythonClient', 'x-global-transaction-id': '7wxe4bd9fb85484554lcmal3t7t4c9chbac1', 'Strict-Transport-Security': 'max-age=31536000; includeSubDomains'})
HTTP response body: {"trace":"-yfu7eisl65xg","errors":[{"code":"library_already_exists","message":"A library already exists that matches with the provided input - name, version and platform. Provide unique values for these fields."}]}

2020-08-18 19:12:56,838 - watson_machine_learning_client.wml_client_err

As próximas células irão realizar o deploy da pipeline declarada neste notebook no WML. Só prossiga se você já está satisfeito com seu modelo e acha que já é a hora de fazer o deploy da sua solução.

Cole as credenciais de sua instância do Watson Machine Learning na variável na célula abaixo.

É importante que a variável que contém os valores tenha o nome de ``wml_credentials`` para que as próximas células deste notebook executem corretamente.

In [ ]:
wml_credentials = {
  "apikey": "x-T5o0RXmA3Ep1UtLG4jz7u6wcp_i0Ms_2wKvQVS6UWz",
  "iam_apikey_description": "Auto-generated for key a0e0989a-2f76-4d4a-8882-499d35091f7c",
  "iam_apikey_name": "Credenciais de serviço",
  "iam_role_crn": "crn:v1:bluemix:public:iam::::serviceRole:Writer",
  "iam_serviceid_crn": "crn:v1:bluemix:public:iam-identity::a/b887b91a23804c7d8a27bba9a139998d::serviceid:ServiceId-c843d73b-73b5-4b31-ab68-0b03f1780cfd",
  "instance_id": "9d01aaf1-612f-457e-9df3-c123eadbd02d",
  "url": "https://eu-de.ml.cloud.ibm.com"
}

In [ ]:
# Instanciando um objeto cliente do Watson Machine Learning a partir das credenciais fornecidas

clientWML = WatsonMachineLearningAPIClient(wml_credentials)

In [ ]:
# Extraindo detalhes da sua instância do Watson Machine Learning

instance_details = clientWML.service_instance.get_details()
print(json.dumps(instance_details, indent=4))

{
    "entity": {
        "source": "Bluemix",
        "published_models": {
            "url": "https://eu-de.ml.cloud.ibm.com/v3/wml_instances/9d01aaf1-612f-457e-9df3-c123eadbd02d/published_models"
        },
        "usage": {
            "capacity_units": {
                "current": 0,
                "limit": 180000000
            },
            "computation_time": {
                "current": 0,
                "limit": 180000
            },
            "deployment_count": {
                "current": 4,
                "limit": 5
            },
            "expiration_date": "2020-09-01T00:00:00.000Z",
            "gpu_count_k80": {
                "current": 0,
                "limit": 8
            },
            "gpu_count_p100": {
                "current": 0,
                "limit": 0
            },
            "gpu_count_v100": {
                "current": 0,
                "limit": 0
            },
            "model_count": {
                "current": 3,
            

**ATENÇÃO!!**

Fique atento para os limites de consumo de sua instância do Watson Machine Learning!

Caso você expire a camada grátis, não será possível avaliar seu modelo (pois é necessária a realização de algumas chamadas de API que consomem predições!)

#### Listando todos os artefatos armazenados no seu WML

Para listar todos os artefatos armazenados em seu Watson Machine Learning, você pode usar a seguinte função:

    clientWML.repository.list()

In [ ]:
# Listando todos os artefatos atualmente armazenados na sua instância do WML

clientWML.repository.list()

------------------------------------  -------------------------------  ------------------------  -----------------  -----------------
GUID                                  NAME                             CREATED                   FRAMEWORK          TYPE
864eb045-9759-44ee-96d0-80b8690393b0  desafio-2-mbtc2020-pipeline-1    2020-08-17T19:20:18.420Z  scikit-learn-0.20  model
48332616-58de-4a38-a08f-ec2223300809  desafio-2-mbtc2020-pipeline-1    2020-08-17T18:18:01.656Z  scikit-learn-0.20  model
ffeca093-e781-4e53-a523-e245c908aac0  desafio-2-mbtc2020-pipeline-1    2020-08-17T18:09:25.454Z  scikit-learn-0.20  model
ea0c0cd9-cc8a-4b42-b2d3-aeae212224f9  desafio-2-mbtc2020-pipeline-1    2020-08-17T17:57:47.049Z  scikit-learn-0.20  model
747f8ccf-2486-4b2f-bbce-35191b629eae  desafio-2-mbtc2020-deployment-1  2020-08-17T19:20:31.733Z  scikit-learn-0.20  online deployment
809e7a70-bca1-4e34-b8c5-f1380872b93a  desafio-2-mbtc2020-deployment-1  2020-08-17T18:18:15.877Z  scikit-learn-0.20  online 

No plano LITE do Watson Machine Learning só é permitido o deploy de um único modelo por vez. Se for o caso de você já possuir um modelo online na sua instância, você pode apagá-lo utilizando o método clientWML.repository.delete():

    artifact_guid = "359c8951-d2fe-4063-8706-cc06b32d5e0d"
    clientWML.repository.delete(artifact_guid)

#### Criando uma nova definição de pacote Python personalizado no WML

O primeiro passo para realizar seu deploy é armazenar o código das transformações personalizadas criadas por você.

Para essa etapa precisamos apenas do arquivo .zip do pacote criado (que já possuimos carregado no Kernel!)

In [ ]:
# Definição de metadados do nosso pacote com as Transforms personalizadas
pkg_meta = {
    clientWML.runtimes.LibraryMetaNames.NAME: "my_custom_sklearn_transform_8",
    clientWML.runtimes.LibraryMetaNames.DESCRIPTION: "A custom sklearn transform",
    clientWML.runtimes.LibraryMetaNames.FILEPATH: "sklearn_transforms.zip",  # Note que estamos utilizando o .zip criado anteriormente!
    clientWML.runtimes.LibraryMetaNames.VERSION: "1.0",
    clientWML.runtimes.LibraryMetaNames.PLATFORM: { "name": "python", "versions": ["3.6"] }
}
custom_package_details = clientWML.runtimes.store_library( pkg_meta )
custom_package_uid = clientWML.runtimes.get_library_uid( custom_package_details )

print("\n Lista de artefatos de runtime armazenados no WML:")
clientWML.repository.list()


 Lista de artefatos de runtime armazenados no WML:
------------------------------------  -------------------------------  ------------------------  -----------------  -----------------
GUID                                  NAME                             CREATED                   FRAMEWORK          TYPE
864eb045-9759-44ee-96d0-80b8690393b0  desafio-2-mbtc2020-pipeline-1    2020-08-17T19:20:18.420Z  scikit-learn-0.20  model
48332616-58de-4a38-a08f-ec2223300809  desafio-2-mbtc2020-pipeline-1    2020-08-17T18:18:01.656Z  scikit-learn-0.20  model
ffeca093-e781-4e53-a523-e245c908aac0  desafio-2-mbtc2020-pipeline-1    2020-08-17T18:09:25.454Z  scikit-learn-0.20  model
ea0c0cd9-cc8a-4b42-b2d3-aeae212224f9  desafio-2-mbtc2020-pipeline-1    2020-08-17T17:57:47.049Z  scikit-learn-0.20  model
747f8ccf-2486-4b2f-bbce-35191b629eae  desafio-2-mbtc2020-deployment-1  2020-08-17T19:20:31.733Z  scikit-learn-0.20  online deployment
809e7a70-bca1-4e34-b8c5-f1380872b93a  desafio-2-mbtc2020-deployment-1  

#### Criando uma nova definição de runtime Python personalizado no WML

O segundo passo é armazenar uma definição de runtime Python para utilizar a nossa biblioteca personalizada.

Isso pode ser feito da seguinte forma:

In [ ]:
runtime_meta = {
    clientWML.runtimes.ConfigurationMetaNames.NAME: "my_custom_wml_runtime_7",
    clientWML.runtimes.ConfigurationMetaNames.DESCRIPTION: "A Python runtime with custom sklearn Transforms",
    clientWML.runtimes.ConfigurationMetaNames.PLATFORM: {
        "name": "python",
        "version": "3.6"
    },
    clientWML.runtimes.ConfigurationMetaNames.LIBRARIES_UIDS: [ custom_package_uid ]
}
runtime_details = clientWML.runtimes.store( runtime_meta )
custom_runtime_uid = clientWML.runtimes.get_uid( runtime_details )

print("\n Detalhes do runtime armazenado:")
print(json.dumps(runtime_details, indent=4))


 Detalhes do runtime armazenado:
{
    "metadata": {
        "guid": "fabee661-4303-45f6-a232-f4ad748d41aa",
        "url": "https://eu-de.ml.cloud.ibm.com/v4/runtimes/fabee661-4303-45f6-a232-f4ad748d41aa",
        "created_at": "2020-08-18T19:12:38.485Z"
    },
    "entity": {
        "name": "my_custom_wml_runtime_7",
        "description": "A Python runtime with custom sklearn Transforms",
        "custom_libraries": [
            {
                "name": "my_custom_sklearn_transform_8",
                "version": "1.0",
                "url": "https://private.eu-de.ml.cloud.ibm.com/v4/libraries/5ea0f688-3eef-493e-8d7d-d4d38fb8acd6"
            }
        ],
        "content_url": "https://private.eu-de.ml.cloud.ibm.com/v4/runtimes/fabee661-4303-45f6-a232-f4ad748d41aa/content",
        "platform": {
            "name": "python",
            "version": "3.6"
        }
    }
}


In [ ]:
# Listando todos runtimes armazenados no seu WML:
clientWML.runtimes.list()

------------------------------------  -----------------------  ------------------------  ----------
GUID                                  NAME                     CREATED                   PLATFORM
fabee661-4303-45f6-a232-f4ad748d41aa  my_custom_wml_runtime_7  2020-08-18T19:12:38.485Z  python-3.6
8b7ff3d9-c023-4f97-9bd9-7e35a7115c31  my_custom_wml_runtime_7  2020-08-17T19:20:12.797Z  python-3.6
9847452a-e5e5-4a6d-9bcc-83c3aee92a8b  my_custom_wml_runtime_4  2020-08-17T18:17:08.976Z  python-3.6
00f0c2b8-84c0-414e-99d0-f0b52976ecf0  my_custom_wml_runtime_4  2020-08-17T18:09:13.809Z  python-3.6
b5363d75-80a8-4acb-b67d-aebebe2ff6a5  my_custom_wml_runtime_4  2020-08-17T17:57:28.704Z  python-3.6
1d95e568-aec4-47cd-90c5-17819b1165e4  my_custom_wml_runtime_3  2020-08-17T17:08:18.758Z  python-3.6
acd4e259-eaeb-4aa6-a3a2-037a8e30ba7c  my_custom_wml_runtime_2  2020-08-17T17:05:19.285Z  python-3.6
066c923b-27e4-46dc-9031-b53e0957c176  my_custom_wml_runtime_1  2020-08-17T17:02:41.846Z  python-3.6
c5

#### Criando uma nova definição de Pipeline personalizada no WML

Finalmente iremos criar uma definição (metadados) para a nossa Pipeline ser hospedada no WML.

Definimos como parâmetros um nome para o artefato e o ID do runtime criado anteriormente.

In [ ]:
model_meta = {
    clientWML.repository.ModelMetaNames.NAME: 'desafio-2-mbtc2020-pipeline-1',
    clientWML.repository.ModelMetaNames.DESCRIPTION: "my pipeline for submission",
    clientWML.repository.ModelMetaNames.RUNTIME_UID: custom_runtime_uid
}

Em seguida chamamos o método para armazenar a nova definição:

In [ ]:
# Função para armazenar uma definição de Pipeline no WML
stored_model_details = clientWML.repository.store_model(
    model=my_pipeline,  # `my_pipeline` é a variável criada anteriormente e contém nossa Pipeline já treinada :)
    meta_props=model_meta,  # Metadados definidos na célula anterior
    training_data=None  # Não altere esse parâmetro
)

print("\n Lista de artefatos armazenados no WML:")
clientWML.repository.list()

# Detalhes do modelo hospedado no Watson Machine Learning
print("\n Metadados do modelo armazenado:")
print(json.dumps(stored_model_details, indent=4))


 Lista de artefatos armazenados no WML:
------------------------------------  -------------------------------  ------------------------  -----------------  -----------------
GUID                                  NAME                             CREATED                   FRAMEWORK          TYPE
96b4905e-4c8e-43ae-a8a5-74d24e43a79f  desafio-2-mbtc2020-pipeline-1    2020-08-18T19:12:44.780Z  scikit-learn-0.20  model
864eb045-9759-44ee-96d0-80b8690393b0  desafio-2-mbtc2020-pipeline-1    2020-08-17T19:20:18.420Z  scikit-learn-0.20  model
48332616-58de-4a38-a08f-ec2223300809  desafio-2-mbtc2020-pipeline-1    2020-08-17T18:18:01.656Z  scikit-learn-0.20  model
ffeca093-e781-4e53-a523-e245c908aac0  desafio-2-mbtc2020-pipeline-1    2020-08-17T18:09:25.454Z  scikit-learn-0.20  model
ea0c0cd9-cc8a-4b42-b2d3-aeae212224f9  desafio-2-mbtc2020-pipeline-1    2020-08-17T17:57:47.049Z  scikit-learn-0.20  model
747f8ccf-2486-4b2f-bbce-35191b629eae  desafio-2-mbtc2020-deployment-1  2020-08-17T19:20:31.733

#### Realizando o deployment do seu modelo para consumo imediato por outras aplicações

In [ ]:
# O deployment do modelo é finalmente realizado por meio do método ``deployments.create()``

model_deployment_details = clientWML.deployments.create(
    artifact_uid=stored_model_details["metadata"]["guid"],  # Não altere esse parâmetro
    name="desafio-2-mbtc2020-deployment-1",
    description="Solução do desafio 2 - MBTC",
    asynchronous=False,  # Não altere esse parâmetro
    deployment_type='online',  # Não altere esse parâmetro
    deployment_format='Core ML',  # Não altere esse parâmetro
    meta_props=model_meta  # Não altere esse parâmetro
)

{"trace":"a42c4a609ce604f58b243ea8201306bc","errors":[{"code":"deployments_plan_limit_reached","message":"Current plan 'lite' only allows 5 deployments"}]}


--------------------------
Deployment creation failed
--------------------------




WMLClientError: Deployment creation failed. Error: 402. {"trace":"a42c4a609ce604f58b243ea8201306bc","errors":[{"code":"deployments_plan_limit_reached","message":"Current plan 'lite' only allows 5 deployments"}]}

#### Testando um modelo hospedado no Watson Machine Learning

In [ ]:
# Recuperando a URL endpoint do modelo hospedado na célula anterior

model_endpoint_url = clientWML.deployments.get_scoring_url(model_deployment_details)
print("A URL de chamada da sua API é: {}".format(model_endpoint_url))

In [ ]:
# Detalhes do deployment realizado

deployment_details = clientWML.deployments.get_details(
    deployment_uid=model_deployment_details["metadata"]["guid"]  # esse é o ID do seu deployment!
)

print("Metadados do deployment realizado: \n")
print(json.dumps(deployment_details, indent=4))

Metadados do deployment realizado: 

{
    "metadata": {
        "guid": "747f8ccf-2486-4b2f-bbce-35191b629eae",
        "url": "https://eu-de.ml.cloud.ibm.com/v3/wml_instances/9d01aaf1-612f-457e-9df3-c123eadbd02d/deployments/747f8ccf-2486-4b2f-bbce-35191b629eae",
        "created_at": "2020-08-17T19:20:31.733Z",
        "modified_at": "2020-08-17T19:20:32.190Z"
    },
    "entity": {
        "runtime_environment": "python-3.6",
        "name": "desafio-2-mbtc2020-deployment-1",
        "scoring_url": "https://eu-de.ml.cloud.ibm.com/v3/wml_instances/9d01aaf1-612f-457e-9df3-c123eadbd02d/deployments/747f8ccf-2486-4b2f-bbce-35191b629eae/online",
        "deployable_asset": {
            "name": "desafio-2-mbtc2020-pipeline-1",
            "url": "https://eu-de.ml.cloud.ibm.com/v3/wml_instances/9d01aaf1-612f-457e-9df3-c123eadbd02d/published_models/864eb045-9759-44ee-96d0-80b8690393b0",
            "guid": "864eb045-9759-44ee-96d0-80b8690393b0",
            "description": "my pipeline for s

In [ ]:
scoring_payload = {
    'fields': [
        "MATRICULA", "NOME", 'REPROVACOES_DE', 'REPROVACOES_EM', "REPROVACOES_MF", "REPROVACOES_GO",
        "NOTA_DE", "NOTA_EM", "NOTA_MF", "NOTA_GO",
        "INGLES", "H_AULA_PRES", "TAREFAS_ONLINE", "FALTAS", 
    ],
    'values': [
        [
            513949,"Marli Quésia de Oliveira",1,1,1,1,4.3,4.0,3.1,4.9,0,3,4,3,
        ]
    ]
}

print("\n Payload de dados a ser classificada:")
print(json.dumps(scoring_payload, indent=4))


 Payload de dados a ser classificada:
{
    "fields": [
        "MATRICULA",
        "NOME",
        "REPROVACOES_DE",
        "REPROVACOES_EM",
        "REPROVACOES_MF",
        "REPROVACOES_GO",
        "NOTA_DE",
        "NOTA_EM",
        "NOTA_MF",
        "NOTA_GO",
        "INGLES",
        "H_AULA_PRES",
        "TAREFAS_ONLINE",
        "FALTAS"
    ],
    "values": [
        [
            513949,
            "Marli Qu\u00e9sia de Oliveira",
            1,
            1,
            1,
            1,
            4.3,
            4.0,
            3.1,
            4.9,
            0,
            3,
            4,
            3
        ]
    ]
}


In [ ]:
result = clientWML.deployments.score(
    model_endpoint_url,
    scoring_payload
)

print("\n Resultados:")
print(json.dumps(result, indent=4))

NameError: name 'model_endpoint_url' is not defined

<hr>

## Parabéns! 

Se tudo foi executado sem erros, você já tem um classificador baseado em machine learning encapsulado como uma API REST!

Para testar a sua solução integrada com um assistente virtual e realizar a submissão, acesse a página:

https://uninassau.maratona.dev

Você irá precisar da endpoint url do seu modelo e das credenciais do WML :)